In [1]:
import numpy as  np
import tensorflow as  tf

#### 设置参数

In [2]:
num_exam=10
num_words=20
num_feat=100
num_tags=5

#### 构建随机特征

In [3]:
x=np.random.rand(num_exam,num_words,num_feat).astype(np.float32)
print("x:",x.shape)
#构建随机tag
y=np.random.randint(num_tags,size=[num_exam,num_words]).astype(np.int32)
print('y:',y.shape)
#获取样本句长
length_se=np.full(num_exam,num_words,dtype=np.int32)
print('length_se:',length_se.shape)

x: (10, 20, 100)
y: (10, 20)
length_se: (10,)


#### 训练模型

In [7]:
with tf.Graph().as_default():
    with tf.Session() as session:
        x_t = tf.constant(x)
        print("x_t:】\t\t\t", x_t)

        y_t = tf.constant(y)
        print("y_t:】\t\t\t", y_t)
        length_se_t = tf.constant(length_se)
        print("length_se_t:】\t\t\t", length_se_t)
        # 增加一个无偏置线性层
        weights = tf.get_variable("weights", [num_feat, num_tags])
        print("weights:】\t\t\t", weights)
        x_t_matr = tf.reshape(x_t, [-1, num_feat])
        print("x_t_matr:】\t\t\t", x_t_matr)
        unary_scores_matr = tf.matmul(x_t_matr, weights)
        print("unary_scores_matr:】\t\t\t", unary_scores_matr)
        unary_scores = tf.reshape(unary_scores_matr, [num_exam, num_words, num_tags])
        print("unary_scores:】\t\t\t", unary_scores)
        # 计算标签序列的log-likelihood并获得转移概率
        log_likelihood, tran_params = tf.contrib.crf.crf_log_likelihood(unary_scores, y_t, length_se)
        print("log_likelihood:】\t\t\t", log_likelihood)
        print("tran_params:】\t\t\t", tran_params)
        # 解码
        viterbi_sequence, viterbi_score = tf.contrib.crf.crf_decode(unary_scores, tran_params, length_se_t)
        print("viterbi_sequence, viterbi_score:】\t", viterbi_sequence, viterbi_score)
        loss = tf.reduce_mean(-log_likelihood)
        train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
        print("train_op:】\t\t\t", train_op)
        session.run(tf.global_variables_initializer())
        mask = (np.expand_dims(np.arange(num_words), axis=0) < np.expand_dims(length_se, axis=1))
        print("mask:\t\t\t", mask)
        # 获取总标签数
        total_labels = np.sum(length_se)
        print("获取总标签数:", total_labels)
        # 开始训练
        for i in range(500):
            tf_viterbi_sequence, _ = session.run([viterbi_sequence, train_op])
            if i % 100 == 0:
                correct_lables = np.sum((y == tf_viterbi_sequence) * mask)
                accuracy = 100.0 * correct_lables / float(total_labels)
                print("Accuracy-NO.%d:%.2f%%" % (i, accuracy))

x_t:】			 Tensor("Const:0", shape=(10, 20, 100), dtype=float32)
y_t:】			 Tensor("Const_1:0", shape=(10, 20), dtype=int32)
length_se_t:】			 Tensor("Const_2:0", shape=(10,), dtype=int32)
weights:】			 <tf.Variable 'weights:0' shape=(100, 5) dtype=float32_ref>
x_t_matr:】			 Tensor("Reshape:0", shape=(200, 100), dtype=float32)
unary_scores_matr:】			 Tensor("MatMul:0", shape=(200, 5), dtype=float32)
unary_scores:】			 Tensor("Reshape_1:0", shape=(10, 20, 5), dtype=float32)
log_likelihood:】			 Tensor("sub_1:0", shape=(10,), dtype=float32)
tran_params:】			 <tf.Variable 'transitions:0' shape=(5, 5) dtype=float32_ref>
viterbi_sequence, viterbi_score:】	 Tensor("ReverseSequence_1:0", shape=(10, 20), dtype=int32) Tensor("Max:0", shape=(10,), dtype=float32)
train_op:】			 name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_weights/ApplyGradientDescent"
input: "^GradientDescent/update_transitions/ApplyGradientDescent"

mask:			 [[ True  True  True  True  True  True  True  True  True  True